# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [13]:
cities_df = pd.read_csv("output_data/cities.csv")
cities_df.head()


,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,Port Elizabeth,-33.9180,25.5701,66.20,88,20,13.80,ZA,2021-03-12 03:03:14
1,Sitka,57.0531,-135.3300,35.60,64,90,6.69,US,2021-03-12 03:03:16
2,Hilo,19.7297,-155.0900,75.20,83,90,9.22,US,2021-03-12 03:03:17
3,Celestún,20.8667,-90.4000,76.69,73,3,17.60,MX,2021-03-12 03:03:18
4,Butaritari,3.0707,172.7902,81.91,74,0,13.24,KI,2021-03-12 03:03:19


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [10]:
locations = cities_df[["Latitude", "Longitude"]]

humidity_as_weight = cities_df["Humidity"]

fig = gmaps.figure(zoom_level=1, center=(0, 0))

# Adding Heatmap layer to map
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_as_weight, 
                                 dissipating=False, max_intensity=200,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

# Narrow down the DataFrame to find your ideal weather condition. For example:
A max temperature lower than 80 degrees but higher than 70.
Wind speed less than 10 mph.
Zero cloudiness.
Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
Using Google Places API to find the first hotel for each city located within 5000 meters of your coordinates.
Plot the hotels on top of the humidity heatmap with each pin containing the Hotel Name, City, and Country.

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [40]:
nice_weather_df = cities_df.loc[(cities_df["Max Temperature"] > 70) & 
                                  (cities_df["Max Temperature"] < 80) & 
                                  (cities_df["Wind Speed"] < 10) &
                                  (cities_df["Cloudiness"] == 0)]

nice_weather_df = nice_weather_df.reset_index(drop=True)

nice_weather_df.count()

City               19
Latitude           19
Longitude          19
Max Temperature    19
Humidity           19
Cloudiness         19
Wind Speed         19
Country            19
Date               19
dtype: int64

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [66]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"
place_type = "lodging"
radius = 5000

hotel_name_list = []
hotel_latitude = []
hotel_longitude = []

for index, row in nice_weather_df.iterrows():
    lat = row["Latitude"]
    lng = row["Longitude"]
    location = (f"{lat},{lng}")
    
    query_url = f"{url}location={location}&radius={radius}&type={place_type}&key={g_key}"
    
    response = requests.get(query_url)
    response_json = response.json()
    
    try:
        hotel_name_list.append(response_json["results"][0]["name"])
        hotel_latitude.append(response_json["results"][0]["geometry"]["location"]["lat"])
        hotel_longitude.append(response_json["results"][0]["geometry"]["location"]["lng"])
        
    except (KeyError, IndexError):
        print("search done")

search done


In [67]:
hotel_name_list

['Crystal Beach and Camping Resort Ormara',
 'Chambres d\'hôte "La Trinité"',
 'Quality Suites Vila Velha',
 'Kashmiri Hotal',
 'Hotel Pigalle',
 'Hotel Manglares',
 'Hotel Gualok',
 'Chácara Sombra do Altíssimo',
 'Adventure Plus Resort & Park.',
 'Pousada Beija Flor',
 'Dharmakar Nagoji sowrakshana complex',
 'Pousada 100 Passos',
 'Pousada Aconchego',
 'Hôtel Cap Macabou',
 'Lords Inn Somnath',
 'Forasteiro Pousada - Hotel',
 'Dream Land',
 'Falang Government Guesthouse']

In [68]:
hotel_df = pd.DataFrame(
    {'Hotel Name' : hotel_name_list,
    'Hotel Latitude' : hotel_latitude,
    'Hotel Longitude' : hotel_longitude,}
)

hotel_df.head()

,Hotel Name,Hotel Latitude,Hotel Longitude
0,Crystal Beach and Camping Resort Ormara,25.235929,64.608533
1,"Chambres d'hôte ""La Trinité""",-21.358434,55.770322
2,Quality Suites Vila Velha,-20.335028,-40.280975
3,Kashmiri Hotal,25.263607,63.470488
4,Hotel Pigalle,-6.970959,-34.837995


In [73]:
hotel_test_df = pd.concat([nice_weather_df, hotel_df], axis=1)
hotel_test_df.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,City,...,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Latitude,Hotel Longitude
0,Ormara,25.2088,64.6357,72.63,79,0,3.51,PK,2021-03-12 03:03:38,Ormara,...,64.6357,72.63,79,0,3.51,PK,2021-03-12 03:03:38,Crystal Beach and Camping Resort Ormara,25.235929,64.608533
1,Saint-Philippe,-21.3585,55.7679,75.20,64,0,6.91,RE,2021-03-12 03:03:56,Saint-Philippe,...,55.7679,75.20,64,0,6.91,RE,2021-03-12 03:03:56,"Chambres d'hôte ""La Trinité""",-21.358434,55.770322
2,Vila Velha,-20.3297,-40.2925,77.00,88,0,5.75,BR,2021-03-12 03:00:24,Vila Velha,...,-40.2925,77.00,88,0,5.75,BR,2021-03-12 03:00:24,Quality Suites Vila Velha,-20.335028,-40.280975
3,Pasni,25.2631,63.4710,73.15,77,0,6.67,PK,2021-03-12 03:06:10,Pasni,...,63.4710,73.15,77,0,6.67,PK,2021-03-12 03:06:10,Kashmiri Hotal,25.263607,63.470488
4,Cabedelo,-6.9811,-34.8339,78.80,89,0,4.61,BR,2021-03-12 03:06:43,Cabedelo,...,-34.8339,78.80,89,0,4.61,BR,2021-03-12 03:06:43,Hotel Pigalle,-6.970959,-34.837995


In [71]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: "None of [Index(['Lat', 'Lng'], dtype='object')] are in the [columns]"

In [ ]:
# Add marker layer ontop of heat map


# Display figure
